In [2]:
import sqlite3
import psycopg2
import pandas as pd
import numpy as np
import math
from DataBase_functions import *
from DataBase_functions_testing import *

#Google sheed API
import gspread
from google.oauth2.service_account import Credentials
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

<p style="font-size: 30px; text-align: center; color: #20a7e5;">Creating data-frame</p>

<p style="font-size: 25px; color: #208ee5">Registro actividad todos</p>

In [2]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    srpeadsheet_id = "1223iwU7YQTMEJ_zjRjWci_E6fDnwP09Bflt4b_iOJzo" 
    #28.12.2024: 12qLRAp7C9qzcoWkyUTJAVMhRHMFfwYec094J8CU2t7c 
    #23.01.2025: 1223iwU7YQTMEJ_zjRjWci_E6fDnwP09Bflt4b_iOJzo 

    # Open the Google Sheet file by ID
    spreadsheet = client.open_by_key(srpeadsheet_id)

    # List all sheets file
    sheets = spreadsheet.worksheets()
    print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

    #Dataframe con los datos de todos los meses
    df = pd.DataFrame(columns=["Fecha de alta", "Fecha incurrida", "Fecha de fin", "UUAA", "Código", "SDATOOL", "Feature", "Petición", "Geografía", "Gestor BBDD", "Ámbito", "Responsable", "Validada", "Horas", "Comentarios"]) 
    meses = ["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre", "Enero_25"]
    for sheet in sheets:
        sheet_name = sheet.title 
        if sheet_name in meses:
            df_aux = pd.DataFrame(sheet.get_all_records(expected_headers=["Fecha de alta", "Fecha incurrida", "Fecha de fin", "UUAA", "Código", "SDATOOL", "Feature", "Petición", "Geografía", "Gestor BBDD", "Ámbito", "Responsable", "Validada", "Horas", "Comentarios"]))
            df = pd.concat([df, df_aux], ignore_index=True)
        else: 
            print(sheet_name, "is not a month")
            pass
    #df = df.drop_duplicates(subset='Código', keep='first')
    print(df.shape, "\n", df.columns)

    #Dataframe with 2024 sheet data
    sheet_2024 = spreadsheet.worksheet("2024")
    df_2024 = pd.DataFrame(sheet_2024.get_all_records(expected_headers=["Fecha de alta", "Fecha incurrida", "Fecha de fin", "UUAA", "Código", "SDATOOL", "Feature", "Petición", "Geografía", "Gestor BBDD", "Ámbito", "Responsable", "Validada", "Horas", "Comentarios"]))
    #df_2024 = df_2024.drop_duplicates(subset='Código', keep='first')
    print(df_2024.shape, "\n", df_2024.columns)

    #Concatenate both dataframes
    df = pd.concat([df, df_2024], ignore_index=True)
    df = df.drop_duplicates(subset="Código", keep="first")
    print(df.shape)

    #Dataframe validation
    df.rename(columns={"Validada": "validada", "Responsable": "responsable", "Gestor DDBB": "DDBB", "Geografía": "geography", "Petición": "petition_code", "Feature": "feature", "SDATOOL": "sdatool", "Código": "DQDP_code", "Gestor BBDD": "DDBB", "Horas": "duration_time", "Comentarios": "description", "Fecha de alta": "fecha_in", "Fecha de fin": "fecha_out"}, inplace=True)

    df["fecha_in"] = df["fecha_in"].astype(str)
    df["Fecha incurrida"] = df["Fecha incurrida"].astype(str)
    df["fecha_out"] = df["fecha_out"].astype(str)
    df["UUAA"] = df["UUAA"].astype(str)
    df["DQDP_code"] = df["DQDP_code"].astype(str)
    df["sdatool"] = df["sdatool"].astype(str)
    df["feature"] = df["feature"].astype(str)
    df["geography"] = df["geography"].astype(str)
    df["DDBB"] = df["DDBB"].astype(str)
    df["Ámbito"] = df["Ámbito"].astype(str)
    df["responsable"] = df["responsable"].astype(str)
    df["validada"] = df["validada"].astype(str)
    df["duration_time"] = pd.to_numeric(df["duration_time"], errors="coerce")
    df["description"] = df["description"].astype(str)
    df["petition_code"] = df["petition_code"].astype(str)

    df["version_date"] = "Nan"
    df["version"] = "Nan"
    df["petition_arq"] = "Nan"
    df["dev_master"] = "Nan"
    df["estado"] = "Nan"

except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

Available sheeds: ['2024 totales - Peticiones', '2024 totales - Peticiones - Pro', '2024', 'Gráficos Semanales - Procesos', '2024 - Procesos', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre', 'Enero_25', 'Responsables', 'None', 'Peticiones totales']
2024 totales - Peticiones is not a month
2024 totales - Peticiones - Pro is not a month
2024 is not a month
Gráficos Semanales - Procesos is not a month
2024 - Procesos is not a month
Responsables is not a month
None is not a month
Peticiones totales is not a month
(4839, 17) 
 Index(['Fecha de alta', 'Fecha incurrida', 'Fecha de fin', 'UUAA', 'Código',
       'SDATOOL', 'Feature', 'Petición', 'Geografía', 'Gestor BBDD', 'Ámbito',
       'Responsable', 'Validada', 'Horas', 'Comentarios', 'Estado', ''],
      dtype='object')
(2141, 18) 
 Index(['Fecha de alta', 'Fecha incurrida', 'Fecha de fin', 'UUAA', 'Código',
       'SDATOOL', 'Feature', 'Petición', 'Geografía', 'Gestor BBDD', 'Ámbito',
    

<p style="font-size: 25px; color: #208ee5; text-align: center;">Peticiones globales Oracle, Elastic Search y Mongo DB</p>

<p style="font-size: 25px; color: #208cc5;;">Oracle Physics</p>

In [3]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    srpeadsheet_id = "1PZKCpybEechpbwP-H3khhUihHqwUqgbi_KkM6iCxhuE" #28.12.2024: "1PZKCpybEechpbwP-H3khhUihHqwUqgbi_KkM6iCxhuE"; 23.01.2025: 1Ia7E6mfBtF6q9PcGwSA8Qa9OR4B8wTB_LjCjJnZo-70

    # Open the Google Sheet file by ID
    spreadsheet = client.open_by_key(srpeadsheet_id)

    # List all sheets file
    sheets = spreadsheet.worksheets()
    print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

    #DataFrame con los datos del excel
    df_OG = pd.DataFrame(columns=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"])

    for sheet in sheets: 
        try: 
            sheet_name = sheet.title
            
            df_aux = pd.DataFrame(sheet.get_all_records(expected_headers=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"]))
            df_OG = pd.concat([df_OG, df_aux], ignore_index=True)
        except gspread.exceptions.APIError as e:
            if e.response.status_code == 429: 
                wait_time = 6
                print(f"Quota exceeded. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else: 
                pass
    
    #Creating DDBB and geography column
    df_OG["DDBB"] = "Oracle Physics"
    df_OG["geography"] = "Global"

    print(df_OG.shape)

    df_OG["UUAA"] = df_OG["UUAA"].astype(str)
    df_OG["Petición Jira"] = df_OG["Petición Jira"].astype(str)
    df_OG["Versión Repo"] = df_OG["Versión Repo"].astype(str)
    df_OG["Petición ARQ"] = df_OG["Petición ARQ"].astype(str)
    df_OG["Fecha cierre"] = df_OG["Fecha cierre"].astype(str)
    df_OG["Comentarios"] = df_OG["Comentarios"].astype(str)
    df_OG["Path"] = df_OG["Path"].astype(str)
    df_OG["DDBB"] = df_OG["DDBB"].astype(str)
    df_OG["geography"] = df_OG["geography"].astype(str)

    df_OG.rename(columns={"Petición Jira": "petition_code", "Versión Repo": "version", "Petición ARQ": "petition_arq", "Fecha cierre": "fecha_out", "Comentarios": "description"}, inplace=True)

except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

Available sheeds: ['APIC', 'ATAU', 'B1DS', 'B43T', 'BJP9', 'CCUS', 'DAMS', 'DMTO', 'EGRP', 'EI4M', 'ESCD', 'FZHI', 'GDEL', 'GHDC', 'GJSX', 'GSIN', 'GUAR', 'H9IY', 'HFPP', 'HLCY', 'HPGR', 'IMPT', 'J6G7', 'K5VI', 'KAPI', 'KARC', 'KBGE', 'KBTQ', 'KCCX', 'KCDO', 'KCMC', 'KCMS', 'KCNC', 'KCNS', 'KCOG', 'KCOL', 'KCSN', 'KCSP', 'KDCL', 'KDFH', 'KDMG', 'KERF', 'KESJ', 'KFGC', 'KFRE', 'KFUL', 'KGCC', 'KGCI', 'KGTP', 'KIFV', 'KIVS', 'KLIM', 'KLNE', 'KMET', 'KMIC', 'KMOL', 'KMRK', 'KMYC', 'KNWD', 'KOMN', 'KONB', 'KPAG', 'KPAD', 'KPAY', 'KPDA', 'KPDR', 'KPFE', 'KPFM', 'KPIG', 'KPPS', 'KPST', 'KRBA', 'KRCE', 'KSAG', 'KSAN', 'KSDD', 'KSPH', 'KSRO', 'KUSU', 'KVCS', 'KXSG', 'KYSZ', 'KYOP', 'KYUB', 'MCRR', 'MFCG ', 'MK0W', 'MDDM', 'NI0C', 'O43T', 'PGVC', 'PIVU', 'PMSC', 'PMSV', 'P6ES', 'QWJR', 'QWPJ', 'RLR7', 'SWP3', 'S47B', 'UGX2', 'V34F', 'VAMO', 'W1BD', 'XBOB', 'ZGR7', 'XMBR', 'XR6Z', 'ZDNM', 'ZNXD']
Quota exceeded. Retrying in 6 seconds...
Quota exceeded. Retrying in 6 seconds...
Quota exceeded. Re

<p style="font-size: 25px; color: #208cc5;;">Elastic Search</p>

In [4]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    srpeadsheet_id = "1PXFKrrJVs12XpmgV5DmnBvjxENQ3hobwLs-bdDf8OIE" #28.01.2024: "1PXFKrrJVs12XpmgV5DmnBvjxENQ3hobwLs-bdDf8OIE"; 23.01.2025: 15Ak0o3xXZ22Xeg36pl10WMPQxQ_50x2IGG7ceUnP1Ec

    # Open the Google Sheet file by ID
    spreadsheet = client.open_by_key(srpeadsheet_id)

    # List all sheets file
    sheets = spreadsheet.worksheets()
    print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

    #DataFrame con los datos del excel
    df_EG = pd.DataFrame(columns=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"])

    for sheet in sheets: 
        try: 
            sheet_name = sheet.title
            
            df_aux = pd.DataFrame(sheet.get_all_records(expected_headers=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"]))
            df_EG = pd.concat([df_EG, df_aux], ignore_index=True)
        except gspread.exceptions.APIError as e:
            if e.response.status_code == 429: 
                wait_time = 6
                print(f"Quota exceeded. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else: 
                pass
    
    #Creating DDBB and geography column
    df_EG["DDBB"] = "Oracle Physics"
    df_EG["geography"] = "Global"

    #df = df_OG.drop_duplicates(subset='Código', keep='first')
    print(df_EG.shape)

    df_EG["UUAA"] = df_EG["UUAA"].astype(str)
    df_EG["Petición Jira"] = df_EG["Petición Jira"].astype(str)
    df_EG["Versión Repo"] = df_EG["Versión Repo"].astype(str)
    df_EG["Petición ARQ"] = df_EG["Petición ARQ"].astype(str)
    df_EG["Fecha cierre"] = df_EG["Fecha cierre"].astype(str)
    df_EG["Comentarios"] = df_EG["Comentarios"].astype(str)
    df_EG["Path"] = df_EG["Path"].astype(str)
    df_EG["DDBB"] = df_EG["DDBB"].astype(str)
    df_EG["geography"] = df_EG["geography"].astype(str)

    df_EG.rename(columns={"Petición Jira": "petition_code", "Versión Repo": "version", "Petición ARQ": "petition_arq", "Fecha cierre": "fecha_out", "Comentarios": "description"}, inplace=True)

except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

Available sheeds: ['ATAU', 'F30V', 'EI4M', 'GHDC', 'GUAR', 'H9IY', 'IMPT', 'J6G7', 'JBSY', 'KAGE', 'KAPI', 'KBTQ', 'KCNS', 'KHR1', 'KIFV', 'KLNE', 'KMOL', 'KREC', 'KTIN', 'KTRA', 'KTR2', 'KXSG', 'KYGU', 'KYOP', 'L1WI', 'NI0C', 'O43T', 'OPEI', 'PGVC', 'S47B', 'XBMR', 'XPI2', 'YM3X', 'ZNXD']
Quota exceeded. Retrying in 6 seconds...
Quota exceeded. Retrying in 6 seconds...
Quota exceeded. Retrying in 6 seconds...
Quota exceeded. Retrying in 6 seconds...
Quota exceeded. Retrying in 6 seconds...
(248, 10)


<p style="font-size: 25px; color: #208cc5;">Mongo DB</p>

In [5]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    srpeadsheet_id = "1-mO_2AFa0J4VBlONUFPuQ7IfTRNxBb3BbPyEg_D-FPs" #28.12.2024: "1-mO_2AFa0J4VBlONUFPuQ7IfTRNxBb3BbPyEg_D-FPs"; 23.01.2025: 1HKsoAlC2Nm-YUI75-mqAQqBKyqPdc9TcKl2NckwpFew

    # Open the Google Sheet file by ID
    spreadsheet = client.open_by_key(srpeadsheet_id)

    # List all sheets file
    sheets = spreadsheet.worksheets()
    print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

    #DataFrame con los datos del excel
    df_MG = pd.DataFrame(columns=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"])

    for sheet in sheets: 
        try: 
            sheet_name = sheet.title
            
            df_aux = pd.DataFrame(sheet.get_all_records(expected_headers=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"]))
            df_MG = pd.concat([df_MG, df_aux], ignore_index=True)
        except gspread.exceptions.APIError as e:
            if e.response.status_code == 429: 
                wait_time = 6
                print(f"Quota exceeded. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else: 
                pass
    
    #Creating DDBB and geography column
    df_MG["DDBB"] = "Oracle Physics"
    df_MG["geography"] = "Global"
    
    print(df_MG.shape)

    df_MG["UUAA"] = df_MG["UUAA"].astype(str)
    df_MG["Petición Jira"] = df_MG["Petición Jira"].astype(str)
    df_MG["Versión Repo"] = df_MG["Versión Repo"].astype(str)
    df_MG["Petición ARQ"] = df_MG["Petición ARQ"].astype(str)
    df_MG["Fecha cierre"] = df_MG["Fecha cierre"].astype(str)
    df_MG["Comentarios"] = df_MG["Comentarios"].astype(str)
    df_MG["Path"] = df_MG["Path"].astype(str)
    df_MG["DDBB"] = df_MG["DDBB"].astype(str)
    df_MG["geography"] = df_MG["geography"].astype(str)

    df_MG.rename(columns={"Petición Jira": "petition_code", "Versión Repo": "version", "Petición ARQ": "petition_arq", "Fecha cierre": "fecha_out", "Comentarios": "description"}, inplace=True)
    
except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

Available sheeds: ['KSKR', 'KPDR', 'F30V', 'KAPI', 'H9IY', 'KPAD', 'CHM8', 'XMBR', 'KCOG', 'KONL', 'KNWD', 'KIFV', 'KIFD', 'KPFM', 'KHR1', 'KARC', 'KATT']
(77, 9)


<p style="font-size: 25px; color: #20cff5;">All in one Data Frame</p>

In [6]:
df_g = pd.DataFrame(columns=list(df.columns))
df_g = pd.concat([df_g, df_OG, df_EG, df_MG], ignore_index=True)

columns_to_use = ["fecha_in", "fecha_out", "UUAA", "DQDP_code", "sdatool", "feature", "geography", "DDBB", "Ámbito", "responsable", "validada", "description", "version_date", "version", "petition_arq", "estado", "petition_code", "duration_time", "dev_master"]

dataframes = [df, df_g]
for column in columns_to_use: 
    for i in dataframes:
        i[column] = i[column].astype(str)

df["duration_time"] = pd.to_numeric(df["duration_time"], errors="coerce")
df["dev_master"] = "None"

df_g["duration_time"] = pd.to_numeric(df_g["duration_time"], errors="coerce")
df_g["dev_master"] = "None"

####################################################################################################
dataframes_using = [df, df_g]
for i in dataframes_using:
    for row in i.iterrows(): 
        for key, value in dict(row[1]).items():
            if value == "Nan" or value == None or value == "" or value == "nan" or value == "None" or value == "none" or value == "NONE" or value == np.nan: 
                if key == "fecha_in" or key == "fecha_out" or key == "Fecha incurrida" or key == "UUAA" or key == "DQDP_code" or key == "sdatool" or key == "feature" or key == "geography" or key == "DDBB" or key == "Ámbito" or key == "responsable" or key == "validada" or key == "description" or key == "version_date" or key == "version" or key == "petition_arq" or key == "estado" or key == "petition_code" or key == "dev_master":
                    i.at[row[0], key] = "None"
                if key == "duration_time":
                    i.at[row[0], key] = np.nan
                else: 
                    pass

df_g.drop_duplicates(subset=["petition_code"], keep="first", inplace=True)
df_g = df_g[df_g["petition_code"] != "None"]

#df = pd.concat([df, df_g], ignore_index=True)

In [7]:
df_definitive = pd.DataFrame(columns=columns_to_use)
petitions = set(df_g["petition_code"]).union(set(df["petition_code"]))
print(len(petitions), df_definitive.columns)
petitions

2415 Index(['fecha_in', 'fecha_out', 'UUAA', 'DQDP_code', 'sdatool', 'feature',
       'geography', 'DDBB', 'Ámbito', 'responsable', 'validada', 'description',
       'version_date', 'version', 'petition_arq', 'estado', 'petition_code',
       'duration_time', 'dev_master'],
      dtype='object')


{'OTRSSD-15759',
 'OTRSSD-14494',
 'OTRSSD-7956',
 'REQ000101684348',
 'OTRSSD-2236',
 'OTRSSD-13818',
 'OTRSSD-17595',
 'SGS-14842',
 'OTRSSD-18472',
 'REQ000100958660',
 'OTRSSD-15128',
 'WO0000101383668',
 'OTRSSD-6717',
 'OTRSSD-5182',
 'OTRSSD-7336',
 'OTRSSD-7003',
 'OTRSSD-11952',
 'OTRSSD-14528',
 'OTRSSD-3474',
 'OTRSSD-15037',
 'OTRSSD-14442',
 'OTRSSD-10974',
 'OTRSSD-4282',
 'REQ000101136826',
 'OTRSSD-12878',
 'OTRSSD-12614',
 'OTRSSD-16122',
 'SGS-7911',
 'SGS-17702',
 'SGS-12509',
 'REQ000101085285',
 'OTRSSD-17062',
 'OTRSSD-15382',
 'KUVA_20240314130638',
 'OTRSSD-12188',
 'OTRSSD-8926',
 'OTRSSD-16248',
 'OTRSSD-14498',
 'OTRSSD-12657',
 'OTRSSD-12068',
 'OTRSSD-16318',
 'OTRSSD-17103',
 'OTRSSD-16361',
 'OTRSSD-15656',
 'REQ000101530533',
 'OTRSSD-12677',
 'OTRSSD-5192',
 'OTRSSD-4804',
 'OTRSSD-10675',
 'OTRSSD-5743',
 'SGS-20844',
 'OTRSSD-16603',
 'OTRSSD-9530',
 'OTRSSD-17737',
 'SMIM_20241107125721',
 'OTRSSD-10901',
 'OTRSSD-13447',
 'SGS-6205',
 'OTRSSD-11570'

In [8]:
for petition in petitions: 
    dic_aux = {i: [] for i in columns_to_use}

    for column in columns_to_use: 
        df_filtered = df[df["petition_code"] == petition]
        df_g_filtered = df_g[df_g["petition_code"] == petition]
        
        if not df_filtered.empty: 
            df_value = df_filtered[column].values[0]
        else: 
            df_value = "None" if column != "duration_time" else None

        if not df_g_filtered.empty: 
            df_g_value = df_g_filtered[column].values[0]
        else: 
            df_g_value = "None" if column != "duration_time" else None
        

        if df_value != "None" and df_value is not None: 
            dic_aux[column].append(df_value)
        elif df_g_value != "None" and df_g_value is not None: 
            dic_aux[column].append(df_g_value)
        else:
            dic_aux[column].append(df_value) #None o "None"

    df_aux = pd.DataFrame(dic_aux)
    df_definitive = pd.concat([df_definitive, df_aux], ignore_index=True)

C:\Users\matia\AppData\Local\Temp\ipykernel_44204\2068147553.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_definitive = pd.concat([df_definitive, df_aux], ignore_index=True)


In [9]:
dfs = [df, df_g, df_definitive]
df_none_count = {i: 0 for i in list(dfs[2].columns)}
for row in dfs[2].iterrows(): 
    for key, value in dict(row[1]).items():
        if value == "None" or value == None or (isinstance(value, float) and np.isnan(value)):
            df_none_count[key] += 1
        else: 
            pass
print(dfs[0].shape, dfs[1].shape, dfs[2].shape)
df_none_count

(1802, 23) (1099, 24) (2415, 19)


{'fecha_in': 949,
 'fecha_out': 301,
 'UUAA': 2,
 'DQDP_code': 690,
 'sdatool': 770,
 'feature': 748,
 'geography': 205,
 'DDBB': 127,
 'Ámbito': 858,
 'responsable': 1244,
 'validada': 1551,
 'description': 1863,
 'version_date': 2415,
 'version': 1319,
 'petition_arq': 1363,
 'estado': 2415,
 'petition_code': 1,
 'duration_time': 734,
 'dev_master': 2415}

In [10]:
df_definitive

,fecha_in,fecha_out,UUAA,DQDP_code,sdatool,feature,geography,DDBB,Ámbito,responsable,validada,description,version_date,version,petition_arq,estado,petition_code,duration_time,dev_master
0,05/06/2024,05/06/2024,KGTP,DQ-DP-006444,44974,WSONEPIV14-2501,Global,ORACLE Physics,NextGEN,Mario,Si,None,None,None,None,None,OTRSSD-15759,0.5,None
1,None,None,KUSU,DQ-DP-006096,44974,WSONEINFR3-7,None,None,None,Mario,Si,None,None,None,None,None,OTRSSD-14494,1.0,None
2,None,22/05/2023,L1WI,None,None,None,Global,Oracle Physics,None,None,None,None,None,8,DATASD-88556,None,OTRSSD-7956,NaN,None
3,28/11/2024,11/12/2024,KIDS,DQ-DP-007430,44953,DEDTETSTS1-732,España,Teradata,Legacy,None,None,TAS000019929954. Una tabla nueva VBE,None,None,None,None,REQ000101684348,4.0,None
4,None,18/07/2022,KBTQ,None,None,None,Global,Oracle Physics,None,None,None,None,None,22,SGS-21461,None,OTRSSD-2236,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2410,None,09/10/20,KUSU,None,None,None,Global,Oracle Physics,None,None,None,None,None,20,None,None,SGS-5979,NaN,None
2411,18/11/2024,26/11/2024,KFGC,DQ-DP-007338,45730,RICCROSCRO-369,Global,Oracle Physics,NextGEN,None,None,Esperando feature; pedida por petición y priva...,None,v1 rechazada,None,None,OTRSSD-18152,2.0,None
2412,10/01/2025,14/01/2025,KBGE,DQ-DP-007574,44970,ADDC-316,Global,Oracle Physics,NextGEN,Matías,None,"v5 dev -->modificación v47 del master, añaden ...",None,None,None,None,OTRSSD-18654,3.0,None
2413,None,14/02/2023,KBTQ,None,None,None,Global,Oracle Physics,None,None,None,None,None,15,DATASD-64492,None,OTRSSD-5865,NaN,None


---
---

In [11]:
'''
for row in df_definitive.iterrows():
    insert_data_testing(dict(row[1]))
'''

'\nfor row in df_definitive.iterrows():\n    insert_data_testing(dict(row[1]))\n'

<p style="font-size: 40px; color: grey;">Testing</p>

In [12]:
conn1 = sqlite3.connect("BBVA_testing.db")
cursor1 = conn1.cursor()
cursor1.execute("PRAGMA foreign_keys = ON") #In sqlite3 foreign keys are disabled by default

#######
cursor1.execute("SELECT UUAA FROM UUAA")
all_uuaa = [x[0] for x in cursor1.fetchall()]

#######
conn1.commit()
cursor1.close()
conn1.close()

In [3]:
conn3 =  psycopg2.connect(
        dbname = "pwd_control_plnk", 
        user = "matublaq",
        password = "SF19KOpSPMl8Ru51ONQ33AHOf0RuZnne", 
        host = "dpg-ctevf3t6l47c73b4jadg-a.oregon-postgres.render.com",
        port = "5432"
)   
cursor3 = conn3.cursor()

##########


##########
conn3.commit()
cursor3.close()
conn3.close()


OperationalError: connection to server at "dpg-ctevf3t6l47c73b4jadg-a.oregon-postgres.render.com" (35.227.164.209), port 5432 failed: SSL connection has been closed unexpectedly
